# Getting articles from (North) Korean Central News Agency

A scrapper for North Korean main propaganda source. This notebook is developed for articles in Russian, but can also be adapted for any other language. If you are interested only in the resulting Russian dataset, here is a link to download it (15000 entries): [download](https://storage.googleapis.com/ml-bucket-isikus/datasets/ru_kcna_20201229.tar.gz).

## Getting articles

**Important**: Please note that it is advised to run this code on your local machine for the following reasons:
1. Not all IP adresses have access to kcna.kp. For example, I could not obtain anything from within Google Colab.
2. Due to some (expected) peculiarities, there are some actions needed to perform in your browser first: visit [kcna.kp](http://www.kcna.kp/kcna.user.home.retrieveHomeInfoList.kcmsf) and select your desired language from the list in the upper corner. Then, get your `JSESSIONID` from Chrome `Developer Tools > Applications > Cookies` (as described [here](https://www.cookieyes.com/how-to-check-cookies-on-your-website-manually/)) and replace the example string below. Although the correct execution seems to depend on `JSESSIONID` not changing, it is probably better not to change your IP while running this notebook.
3. You should also check if the article count has surpassed 150000 (since on 29 December 2020 the biggest ID was 0140961, I decided to set 15000 as the upper boundary). If it has, change the `maxcount` variable below accordingly.

In [ ]:
jsessionid = "72757373696177696C6C626566726565"  # an example string, you should change it to your JSESSIONID

In [ ]:
import os
import requests
from tqdm.notebook import tqdm

from multiprocessing import Pool

This is where the article data will be saved as separate html files

In [ ]:
!touch missed_articles.txt
!mkdir articles
%cd articles

fails = open("../missed_articles.txt", "a+")

In [ ]:
def fetch_url(entry):
    sid = str(entry)
    sid = sid.rjust(7, '0')
    try:
        _ = !curl 'http://www.kcna.kp/kcna.user.article.retrieveArticleInfoFromArticleCode.kcmsf' \
          -H 'Connection: keep-alive' \
          -H 'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36' \
          -H 'Content-Type: application/x-www-form-urlencoded' \
          -H 'Accept: */*' \
          -H 'Origin: http://www.kcna.kp' \
          -H 'Referer: http://www.kcna.kp/kcna.user.article.retrieveNewsViewInfoList.kcmsf' \
          -H 'Accept-Language: ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7' \
          -H 'Cookie: JSESSIONID={jsessionid}' \
          --data-raw 'article_code=AR{sid}&kwContent=' \
          --compressed \
          --insecure --output {sid}.html
        
        if os.path.isfile(sid+".html"):
            with open(sid+".html", "r", encoding="utf-8") as inr:
                res = inr.read()
        
            if "Connection refused" in res or "We couldn't find your document" in res:
                os.remove(sid+".html")
    except:
        print("Failed at", str(entry))
        fails.write(str(entry)+"\n")
    
    return entry

The code will run around 4 hours. However, the actual data (not blank ID's) will be downloaded roughly during a 1.5 hour period, as at the time of the running the lowest and highest available IDs turned out to be 81871 and 140961, accordingly (for articles in Russian)

In [ ]:
maxcount = 150000  # set a higher value based on current kcna.kp data

try:
    lrange = list(reversed(list(range(maxcount))))

    with Pool(64) as p:
        r = list(tqdm(p.imap(fetch_url, lrange), total=len(lrange), mininterval=10.0))
    fails.close()
except Exception as e:
    fails.close()
    raise RuntimeError(str(e)+"\nOccurred at entry = "+str(results))

## Processing articles

Processing the files using `lxml`

In [ ]:
import re
import os
from datetime import datetime

import pandas as pd
from lxml import html
from tqdm.notebook import tqdm

In [ ]:
entryslist = []

for fname in tqdm(list(os.listdir("articles"))):
    with open("./articles/"+fname, "r", encoding="utf-8") as infile:
        fc = infile.read()

    fc = re.sub(r"<!\[CDATA\[(.*?)\]\]>", r"\1", fc, flags=re.DOTALL)

    tree = html.fromstring(str.encode(fc))
    entry_dict = {}

    entry_dict["article_code"] = str(tree.xpath('//articlecode/text()')[0])
    entry_dict["news_date"] = datetime.strptime(str(tree.xpath('//newsdate/text()')[0]), "%Y%m")
    entry_dict["create_date"] = datetime.strptime(str(tree.xpath('//articlecreatedate/text()')[0]), "%Y.%m.%d")
    entry_dict["disp_title"] = str(tree.xpath('//disptitle/text()')[0])
    entry_dict["main_title"] = str(tree.xpath('//maintitle/text()')[0])
    entry_dict["content"] = str(tree.xpath('//content')[0].text_content())
    
    entryslist.append(entry_dict)

kcna_ru_df = pd.DataFrame(entryslist)

I noted that the strange article pseudo-XML contains two distinct fields for title, which, however, often contain the same string. Here I manually cleaned the data:

In [ ]:
irreg_ac = []

for ac, dt, mt in tqdm(zip(list(kcna_ru_df["article_code"]), list(kcna_ru_df["disp_title"]), list(kcna_ru_df["main_title"]))):
    if dt != mt:
        irreg_ac.append(ac)


for ac, dt, mt in zip(list(kcna_ru_df.loc[kcna_ru_df['article_code'].isin(irreg_ac)]["article_code"]), list(kcna_ru_df.loc[kcna_ru_df['article_code'].isin(irreg_ac)]["disp_title"]), list(kcna_ru_df.loc[kcna_ru_df['article_code'].isin(irreg_ac)]["main_title"])):
    print("ac:", ac)
    print("dt:", dt)
    print("mt:", mt)
    print()

ac: AR0089146
dt: Подписано соглашение об устаножественнывлении друх отношений между университетом имени 
mt: Подписано соглашение об установлении дружественных отношений между университетом имени 

ac: AR0090960
dt: По случаю дня Независимости Государства Катар председатель президиума ВНС КНДР Ким Ен Нам послал поздравительную телеграмму Шейху Государства Катар
mt: По случаю дня Независимости Государства Катар председатель президиума ВНС КНДР Ким Ен Нам послал поздравительную телеграмму Шейху Государства Катар 

ac: AR0091830
dt: Делегация Кхмерской Демократической партии Камбоджи нанесла поздравительный визит в посольство КНДР
mt: анеслаДелегация Кхмерской Демократической партии Камбоджи н поздравительный визит в посольство КНДР

ac: AR0117593
dt: Уважаемый высший руководитель товарищ 
mt: Отбытие президента Мун Чжэ Ина и его сопровождающих из Самчжиёна

ac: AR0134003
dt: Широкое распространение форм сезонных одежд по вкусу женщин--В Дэсонском ателье при Универсальном ателье Дэсонско

In [ ]:
patches = {
    "AR0089146": "Подписано соглашение об установлении дружественных отношений между университетом имени ",
    "AR0090960": "По случаю дня Независимости Государства Катар председатель президиума ВНС КНДР Ким Ен Нам послал поздравительную телеграмму Шейху Государства Катар",
    "AR0091830": "Делегация Кхмерской Демократической партии Камбоджи нанесла поздравительный визит в посольство КНДР",
    "AR0117593": "Отбытие президента Мун Чжэ Ина и его сопровождающих из Самчжиёна",
    "AR0134003": "Широкое распространение форм сезонных одежд по вкусу женщин",
    "AR0136622": "Плохая прелюдия полного провала межкорейских отношений",
    "AR0136854": "Доклад об исследовании Института разоружения и мира МИД КНДР",
    "AR0137433": "Герои-бойцы, бессмертные на лоне великой партии",
    "AR0137941": "Прошел 6-й пленум ЦК ТПК 7-го созыва",
    "AR0139570": "Олимпийские чемпионы в памяти нашего народа (6)"
}

tlist = []

for ac, dt in zip(list(kcna_ru_df["article_code"]), list(kcna_ru_df["disp_title"])):
    if ac in patches:
        tlist.append(patches[ac])
    else:
        tlist.append(dt)

kcna_ru_df["title"] = tlist

Finalizing the collection, saving it as CSV and zipping it:

In [ ]:
kcna_ru_df = kcna_ru_df.drop(["disp_title", "main_title"], axis=1)
kcna_ru_df = kcna_ru_df[["article_code", "news_date", "create_date", "title", "content"]]
kcna_ru_df.sample(7)

,article_code,news_date,create_date,title,content
231,AR0090428,2016-12-01,2016-12-09,Организация ДРК : вся жизнь товарища,"Пхеньян, 7 декабря. /ЦТАК/ -- 2-го декабря..."
405,AR0092008,2017-01-01,2017-01-13,Еще больше знаменитых изделий и товаров,"Пхеньян, 13 января. /ЦТАК/-- На Тэдонганск..."
6793,AR0115870,2018-08-01,2018-08-08,Общереспубликанская презентация научно-техниче...,"Пхеньян, 8 августа. /ЦТАК/-- Общереспубликанск..."
10485,AR0126315,2019-06-01,2019-06-19,Состоялось годичное общее собрание Пэктусанско...,"Пхеньян, 19 июня. /ЦТАК/-- Годичное общее собр..."
12828,AR0133968,2020-03-01,2020-03-02,Газета «Нодон синмун» призвала всех подняться ...,"Пхеньян, 2 марта. /ЦТАК/-- Газета «Нодон синму..."
13181,AR0135303,2020-04-01,2020-04-20,Уважаемый высший руководитель товарищ,"Пхеньян, 20 апреля. /ЦТАК/-- Председатель Госс..."
4811,AR0109894,2018-02-01,2018-02-25,Зампредседателя центрального президиума Чхонрё...,"Пхеньян, 24 февраля. /ЦТАК/-- 23-го февраля за..."


In [ ]:
kcna_ru_df.to_csv("kcna_ru.csv", index=False)

lang = "ru"
fname = lang + "_kcna_" + datetime.now().strftime("%Y%m%d")
!tar -czf {fname}.tar.gz kcna_ru.csv